In [11]:
%pip install --upgrade langchain langchain-ollama langchain-community chromadb
%pip install fastapi nest-asyncio pyngrok uvicorn line-bot-sdk python-dotenv
%pip install fastapi uvicorn python-multipart requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime
import os

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, Request, HTTPException
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

หั่นคำใส่ LLM

In [2]:
def load_documents(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            documents.extend(loader.load())
    return documents

docs = load_documents("./")

In [ ]:
# llm = OllamaLLM(
#     model="scb10x/llama3.2-typhoon2-1b-instruct",
#     temperature=0.3,
# )

# embeddings = OllamaEmbeddings(
#     model="scb10x/llama3.2-typhoon2-1b-instruct"
# )

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=800,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", " ", ""]
# )

# split_docs = text_splitter.split_documents(docs)

# vectorstore = Chroma.from_documents(
#     split_docs,
#     embeddings,
#     collection_name="my_collection"  # ไม่ต้องใส่ persist_directory
# )
# retriever = vectorstore.as_retriever()

In [3]:
llm = OllamaLLM(
    model="llama3.2",
    temperature=0.3,
)

embeddings = OllamaEmbeddings(
    model="llama3.2"
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

split_docs = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    split_docs,
    embeddings,
    persist_directory="./chroma_db"  # กำหนด path db
)

vectorstore.persist()  # save db

retriever = vectorstore.as_retriever()

/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_17251/3085675071.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()  # save db


In [4]:
prompt_template = """
คุณคือผู้ให้คำปรึกษาที่ชื่อว่า "น้องไตน้อย" ผู้ช่วยให้คำปรึกษาที่มีความรู้ความเข้าใจลึกซึ้งในเรื่องการดูแลสุขภาพของผู้ป่วยโรคไตโดยเน้นไปทางด้านอาหารการกิน การคำนวณสารอาหาร คุณสามารถตอบคำถามเกี่ยวกับโรคไต การดูแลสุขภาพได้ 
ตอบคำถามต่อไปนี้โดยยึดตามข้อมูลที่ให้มาเท่านั้น และควรแน่ใจว่าคำตอบถูกต้องตามข้อมูลที่มีอยู่ โดยจะมีข้อมูลผู้ใช้แนบมา โดยความหมายตัวแปรต่าง ๆ จะอยู่ในข้อมูลที่ให้ไป โดยแต่ละตัวแปรจะมีความหมายของมันซึ่งจะเป็นข้อมูลของผู้ใช้ meal_plans ไม่จำเป็นต้องใช้ในการตอบคำถามทุกคำถาม
และคุณต้องเอาข้อมูล "ricipe_name" ไปให้ user เมื่อ user ถามเรื่องกินอะไรดี ถ้าไม่มั่นใจว่ามีข้อมูลก็ควรบอกว่า "ขออภัยไม่สามารถตอบคำถามนี้ได้ คำถามของคุณจะถูกบันทึกไว้เพื่อปรับปรุงคุณภาพของเรา"
ใส่อีโมจิต่าง ๆ ในแต่ละคำตอบด้วย เพื่อความ Friendly มากขึ้น เช่น "✨", "👋"  
และตอบด้วยความชัดเจน กระชับ ในภาษาไทย ลงท้ายด้วยคำว่า "ครับ" เพื่อให้สุภาพ และแทนตัวเองด้วย "น้องไตน้อย" ให้ตอบคำถามให้กระชับ สั้น ๆ และเข้าใจง่ายมากที่สุด


บริบท: {context}

คำถาม: {question}


คำตอบ:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [5]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

def ask_question(data):
    result = qa_chain.invoke({"query": data})
    return result["result"]

In [6]:
question = "โรคไตผมต้องระวังอะไรบ้าง"

data = f"{question}"
answer = ask_question(data)
print(f"คำถาม: {question}")
print(f"คำตอบ: {answer}")

คำถาม: โรคไตผมต้องระวังอะไรบ้าง
คำตอบ: 👋 สวัสดีครับ! น้องไตน้อยที่จะช่วยเหลือคุณเรื่องโรคไตผม 🤔

โรคไตผมเป็นภาวะที่เกิดจากการที่เลือดไม่สามารถทำหน้าที่กำจัดสารพิษออกจากร่างกายได้ตามปกติ ซึ่งอาจทำให้เกิดอาการบวมน้ำหรือภาวะหัวใจล้มเหลวได้

ดังนั้น คุณควรระวังเรื่องของ:

1. **การบริโภคโซเดียม** 🍲: ควรจำกัดโซเดียมในอาหารเพื่อลดความเสี่ยงต่อโรคไตผม
2. **การบริโภคฟอสฟอรัส** 🥗: ควรลดการบริโภคฟอสฟอรัสหากค่าฟอสฟอรัสในเลือดเริ่มสูง
3. **การดื่มน้ำ** 💧: ควรดื่มน้ำเพียงพอเพื่อให้ร่างกายได้รับน้ำที่จำเป็น
4. **การควบคุมน้ำหนัก** 🏋️‍♀️: ควรควบคุมน้ำหนักให้เหมาะสมเพื่อลดความเสี่ยงต่อโรคไตผม

และอีกอย่างหนึ่ง คุณควรปรึกษากับแพทย์หรือผู้เชี่ยวชาญด้านสุขภาพเพื่อให้ได้รับการแนะนำที่เหมาะสมกับสถานการณ์ของคุณ 🤝

หวังว่าคำตอบนี้จะช่วยให้คุณเข้าใจเรื่องโรคไตผมได้ดีขึ้นครับ! 😊


FastAPI เรียกใช้กรณีรูป

In [7]:
from fastapi import FastAPI, File, UploadFile
from dotenv import load_dotenv
import requests
import shutil


load_dotenv()
app = FastAPI()



AI_API_URL = f"https://ai-detect-1025044834972.us-central1.run.app/detect-foods/"

@app.post("/upload/")
async def upload_image(file: UploadFile = File(...)):

    file_location = f"temp/{file.filename}"
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)


    with open(file_location, "rb") as image_file:
        response = requests.post(AI_API_URL, files={"file": image_file})

    return response.json()


เรียก Line 

In [8]:
from dotenv import load_dotenv
from pyngrok import ngrok
import os
import json

load_dotenv()
app = FastAPI()

line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))
ngrok.set_auth_token(os.getenv('NGROK_AUTH_TOKEN'))

/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_17251/2831993446.py:9: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_17251/2831993446.py:10: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))


In [9]:
import requests
import os
from linebot.models import MessageEvent, ImageMessage
from fastapi import FastAPI
import json
from linebot.models import FlexSendMessage, BubbleContainer, CarouselContainer, TextComponent, ButtonComponent, URIAction, PostbackEvent, PostbackAction

def generate_select_recipe_flex(recipes):
    bubbles = []

    for recipe in recipes[:12]:
        bubble = {
    "type": "bubble",
    "size": "micro",
    "body": {
        "type": "box",
        "layout": "vertical",
        "spacing": "sm",
        "paddingAll": "13px",
        "contents": [
            {
                "type": "text",
                "text": recipe["recipes_name"],
                "wrap": True,
                "weight": "bold",
                "size": "sm",
                "color": "#432C7A"
            },
        ]
    },
    "footer": {
        "type": "box",
        "layout": "vertical",
        "contents": [
            {
                "type": "button",
                "style": "primary",
                "color": "#FFA500",
                "action": {
                    "type": "postback",
                    "label": "เลือกเมนูนี้",
                    "data": f"SELECT_RECIPE_{recipe['recipes_id']}"
                },
                "height": "sm"
            }
        ],
        "paddingAll": "13px"
    },
    "styles": {
        "body": {
            "backgroundColor": "#FFF4E0"
        },
        "footer": {
            "separator": True
        }
    }
}
        bubbles.append(bubble)

    flex_message = FlexSendMessage(
        alt_text="เลือกเมนูอาหารที่คุณต้องการ",
        contents={
            "type": "carousel",
            "contents": bubbles
        }
    )

    return flex_message


def generate_flex_message(food_detail):
    return FlexSendMessage(
        alt_text="ข้อมูลอาหาร",
        contents={
    "type": "bubble",
    "header": {
        "type": "box",
        "layout": "vertical",
        "contents": [
            {
                "type": "text",
                "text": "รายละเอียดอาหารที่คุณเลือก",
                "color": "#ffffff",
                "weight": "bold",
                "size": "sm"
            }
        ],
        "backgroundColor": "#FF7E2E",
        "paddingTop": "12px",
        "paddingBottom": "12px",
        "paddingStart": "16px",
        "paddingEnd": "16px"
    },
    "hero": {
        "type": "image",
        "url": food_detail['image_url'][0],
        "size": "full",
        "aspectRatio": "20:13",
        "aspectMode": "cover",
        "action": {
            "type": "uri",
            "uri": f"https://kidney-care.vercel.app/fooddetail/{food_detail['id']}"
        }
    },
    "body": {
        "type": "box",
        "layout": "vertical",
        "backgroundColor": "#FFFFFF",
        "spacing": "md",
        "paddingAll": "16px",
        "contents": [
            {
                "type": "text",
                "text": food_detail['recipe_name'],
                "weight": "bold",
                "size": "xl",
                "wrap": True,
                "color": "#FF7E2E"
            },
            {
                "type": "separator",
                "margin": "lg",
                "color": "#EEEEEE"
            },
            {
                "type": "box",
                "layout": "vertical",
                "margin": "lg",
                "contents": [
                    {
                        "type": "text",
                        "text": "คุณค่าทางโภชนาการ",
                        "weight": "bold",
                        "color": "#555555",
                        "size": "md"
                    },
                    {
                        "type": "box",
                        "layout": "vertical",
                        "margin": "md",
                        "spacing": "sm",
                        "cornerRadius": "md",
                        "backgroundColor": "#F7F7F7",
                        "paddingAll": "12px",
                        "contents": [
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/5787/5787016.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "แคลอรี่",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['calories']} กิโลแคลอรี่",
                                        "color": "#EF5350",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ]
                            },
                            {
                                "type": "separator",
                                "margin": "sm",
                                "color": "#EEEEEE"
                            },
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/2738/2738730.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "โปรตีน",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['protein']} ก.",
                                        "color": "#79A3FF",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ],
                                "margin": "sm"
                            },
                            {
                                "type": "separator",
                                "margin": "sm",
                                "color": "#EEEEEE"
                            },
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/4264/4264699.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "ไขมัน",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['fat']} ก.",
                                        "color": "#FF8568",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ],
                                "margin": "sm"
                            },
                            {
                                "type": "separator",
                                "margin": "sm",
                                "color": "#EEEEEE"
                            },
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/4139/4139391.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "โซเดียม",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['sodium']} มก.",
                                        "color": "#66BB6A",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ],
                                "margin": "sm"
                            },{
                                "type": "separator",
                                "margin": "sm",
                                "color": "#EEEEEE"
                            },
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/2413/2413578.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "คาร์โบไฮเดรค",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['carbs']} ก.",
                                        "color": "#66BB6A",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ],
                                "margin": "sm"
                            },{
                                "type": "separator",
                                "margin": "sm",
                                "color": "#EEEEEE"
                            },
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/3145/3145276.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "ฟอสฟอรัส",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['phosphorus']} ก.",
                                        "color": "#BD7DFF",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ],
                                "margin": "sm"
                            },{
                                "type": "separator",
                                "margin": "sm",
                                "color": "#EEEEEE"
                            },
                            {
                                "type": "box",
                                "layout": "baseline",
                                "contents": [
                                    {
                                        "type": "icon",
                                        "url": "https://cdn-icons-png.flaticon.com/512/2773/2773131.png",
                                        "size": "sm",
                                        "offsetTop": "2px"
                                    },
                                    {
                                        "type": "text",
                                        "text": "โพแทสเซียม",
                                        "color": "#777777",
                                        "size": "sm",
                                        "weight": "bold",
                                        "flex": 2,
                                        "margin": "sm"
                                    },
                                    {
                                        "type": "text",
                                        "text": f"{food_detail['potassium']} ก.",
                                        "color": "#BD7DFF",
                                        "size": "sm",
                                        "flex": 3,
                                        "weight": "bold",
                                        "align": "end"
                                    }
                                ],
                                "margin": "sm"
                            }
                            
                        ]
                    }
                ]
            }
        ]
    },
    "footer": {
        "type": "box",
        "layout": "vertical",
        "spacing": "sm",
        "contents": [
            {
                "type": "button",
                "style": "primary",
                "color": "#FF7E2E",
                "action": {
                    "type": "uri",
                    "label": "ดูรายละเอียดเพิ่มเติม",
                    "uri": f"https://kidney-care.vercel.app/fooddetail/{food_detail['id']}"
                },
                "height": "sm"
            }
        ],
        "paddingAll": "16px"
    },
    "styles": {
        "header": {
            "separator": False
        },
        "body": {
            "separator": False
        },
        "footer": {
            "separator": True
        }
    }
}
    )


app = FastAPI()


# โหลดกำลังพิมพ์
def start_loading_animation(user_id):
    url = "https://api.line.me/v2/bot/chat/loading/start"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))}"
    }
    data = {"chatId": user_id}
    
    response = requests.post(url, headers=headers, json=data, timeout=10)



# โหลดข้อมูลจาก Rust Sqlx
def loading_userInformation(user_id):
    rust_server_url = "https://backend-billowing-waterfall-4640.fly.dev/chatbot/"
    try:
        response = requests.get(f"{rust_server_url}{user_id}",timeout = 7)
        response.raise_for_status()  
        global user_information
        user_information = response.text
        print(f"user_information")
        return response.json()
    
    except requests.exceptions.Timeout:
        print(f"การเชื่อมต่อกับ Rust server หมดเวลา")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ sqlx: {e}")
        return None 


@app.post("/callback")
async def callback(request: Request):
    signature = request.headers['X-Line-Signature']
    body = await request.body()

    try:
        handler.handle(body.decode("utf-8"), signature)
    except InvalidSignatureError:
        raise HTTPException(status_code=400, detail="Invalid signature")

    return 'OK'



# AI Zone

# แก้ให้ query ผ่าน API Food
def get_name(FoodID):
    rust_server_url = "https://backend-billowing-waterfall-4640.fly.dev/food_details/"
    try:
        response = requests.get(f"{rust_server_url}{FoodID}",timeout = 7)
        response.raise_for_status()  
        global food_information
        food_information = response.text
        return response.json()

    except requests.exceptions.Timeout:
        print(f"การเชื่อมต่อกับ Rust server หมดเวลา")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ sqlx: {e}")
        return None 
    

def get_mael_plan(data):
    rust_server_url = "https://backend-billowing-waterfall-4640.fly.dev/get_meal_plan"
    try:
        response = requests.post(
            rust_server_url, 
            json=data,  
            timeout=7
        )
        response.raise_for_status()
        
        global meal_plan_user
        meal_plan_user = response.text
        return response.json()

    except requests.exceptions.Timeout:
        print("การเชื่อมต่อกับ Rust server หมดเวลา")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการเชื่อมต่อ sqlx: {e}")
        return None

@handler.add(PostbackEvent)
def handle_postback(event):
    
    data = event.postback.data

    if data.startswith("SELECT_RECIPE_"):
        user_id = event.source.user_id  
        FoodID = int(data.replace("SELECT_RECIPE_", ""))
        start_loading_animation(user_id)  
        food_detail = get_name(FoodID)
        flex_msg = generate_flex_message(food_detail)

        line_bot_api.reply_message(
            event.reply_token,
            flex_msg
        )

# ดึงข้อมูลจาก AI หลังอัพรูป
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id

    user_id = event.source.user_id  
    start_loading_animation(user_id)  

    image_content = line_bot_api.get_message_content(message_id)

    image_path = f"temp_{message_id}.jpg"
    with open(image_path, "wb") as f:
        for chunk in image_content.iter_content():
            f.write(chunk)

    # ส่งไปยัง Roboflow 
    try:
        with open(image_path, "rb") as img:
            response = requests.post(AI_API_URL, files={"file": img}, timeout = 20)

        result = response.json()
        recipes = result.get("recipes", [])
        print("response", recipes)
        
        if recipes:
            Flex_msg = generate_select_recipe_flex(recipes)

            line_bot_api.reply_message(
                event.reply_token,
                [
                    TextMessage(text="กรุณาเลือกเนื้อสัตว์ของอาหารของคุณ"),
                    Flex_msg
                ]
            )
            # print("เจอ", FoodID)
        else :
            line_bot_api.reply_message (
                event.reply_token,
                TextMessage(text="ขอโทษด้วยเราไม่สามารถตรวจจับรูปที่ไม่ใช่อาหารได้")
            )


        

    except Exception as e:
        print(f"err : {e}")

    finally:
        if os.path.exists(image_path):
            os.remove(image_path)  

    




    
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id  
    print(f"User ID: {user_id}")
    start_loading_animation(user_id)  
    loading_userInformation(user_id)
    data = {
        "user_line_id": user_id,
        "date": datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    }
    get_mael_plan(data)
    
    text = event.message.text


    try:
        data = f"{user_information} {text} {meal_plan_user}"
        if user_information == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text="ขออภัย Server เกิดข้อผิดพลาด กรุณาลองใหม่อีกครั้ง")
            )
        else:
            reply_text = ask_question(data)
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text=reply_text)
            )
        

    except Exception as e:
        print(f"Error in handle_message: {e}")
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="เกิดข้อผิดพลาด กรุณาลองใหม่อีกครั้ง")
        )


def ask_question_with_timeout(data, timeout=20):
    try:
        # ถ้า ask_question เป็น API call หรือกระบวนการที่ใช้เวลานาน
        # ใช้ threading หรือ asyncio เพื่อกำหนด timeout
        from concurrent.futures import ThreadPoolExecutor
        from concurrent.futures import TimeoutError
        
        with ThreadPoolExecutor() as executor:
            future = executor.submit(ask_question, data)
            try:
                return future.result(timeout=timeout)
            except TimeoutError:
                return "ขออภัย การประมวลผลใช้เวลานานเกินไป กรุณาลองใหม่อีกครั้ง"
    except Exception as e:
        print(f"Error in ask_question: {e}")
        return "เกิดข้อผิดพลาดในการประมวลผล กรุณาลองใหม่อีกครั้ง"
ngrok_tunnel = ngrok.connect(8082)
print('Webhook URL:', ngrok_tunnel.public_url + '/callback')

nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8082)



INFO:     Started server process [17251]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8082 (Press CTRL+C to quit)


Webhook URL: https://a7ec-2001-2d8-621a-ee70-118f-5afd-bbc7-d3ae.ngrok-free.app/callback


t=2025-04-10T12:52:30+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=b04502bb18f6 err="read tcp [2001:2d8:621a:ee70:118f:5afd:bbc7:d3ae]:60211->[2600:1f16:d83:1201::6e74:4]:443: read: no route to host"
t=2025-04-10T12:52:30+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-04-10T12:52:31+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-04-10T12:52:32+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-04-10T12:52:34+0900 lvl=eror msg="heartbeat timeout,